In [ ]:
## Import necessary packages
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin

In [2]:
## Part 1

# Obtain and input into a Pandas dataframe the title (column 1), price (column 2), 
# and the number of stars in the rating (column 3) of the first 20 books that show up when you open up the website.

url = "https://books.toscrape.com/"
response = rq.get(url)
soup = bs(response.content, 'html.parser')

titles = []
prices = []
ratings = []

books = soup.find_all('article', class_='product_pod')[:20]

for book in books:
    titles.append(book.h3.a['title'])
    prices.append(float(book.find('p', class_='price_color').text[1:]))
    ratings.append(book.find('p', class_='star-rating')['class'][1])


df = pd.DataFrame({'Title': titles, 'Price': prices, 'Stars': ratings})
display(df)


,Title,Price,Stars
0,A Light in the Attic,51.77,Three
1,Tipping the Velvet,53.74,One
2,Soumission,50.10,One
3,Sharp Objects,47.82,Four
4,Sapiens: A Brief History of Humankind,54.23,Five
5,The Requiem Red,22.65,One
6,The Dirty Little Secrets of Getting Your Dream...,33.34,Four
7,The Coming Woman: A Novel Based on the Life of...,17.93,Three
8,The Boys in the Boat: Nine Americans and Their...,22.60,Four
9,The Black Maria,52.15,One


In [203]:
# Part 2
# Using a loop, complete Part 1 again but for each type of book. That is, create 51 separate 
# dataframes of those three columns for Travel, Mystery,..., Crime.

catagories = soup.find('ul', class_='nav nav-list').find('ul').find_all('a')

dfs = {}

for cat in catagories:
    name = cat.text
    href = url + cat['href']
    cat_resp = rq.get(href)
    cat_soup = bs(cat_resp.content, 'html.parser')
    
    cat_titles = []
    cat_prices = []
    cat_ratings = []
    
    cat_books = cat_soup.find_all('article', class_='product_pod')[:20]
    for book in cat_books:
        cat_titles.append(book.h3.a['title'])
        cat_prices.append(float(book.find('p', class_='price_color').text[1:]))
        cat_ratings.append(book.find('p', class_='star-rating')['class'][1])


    cat_df = pd.DataFrame({'Title': cat_titles, 'Price': cat_prices, 'Stars': cat_ratings})
    dfs[name] = cat_df

In [205]:
for category_name, dff in dfs.items():
    print(f"Category: {category_name}")
    print(dff)
    print()

Category: 
                            
                                Travel
                            
                        
                                                Title  Price  Stars
0                             It's Only the Himalayas  45.17    Two
1   Full Moon over Noah’s Ark: An Odyssey to Mount...  49.43   Four
2   See America: A Celebration of Our National Par...  48.87  Three
3   Vagabonding: An Uncommon Guide to the Art of L...  36.94    Two
4                                Under the Tuscan Sun  37.33  Three
5                                  A Summer In Europe  44.34    Two
6                            The Great Railway Bazaar  30.54    One
7                    A Year in Provence (Provence #1)  56.88   Four
8   The Road to Little Dribbling: Adventures of an...  23.21    One
9           Neither Here nor There: Travels in Europe  38.95  Three
10                 1,000 Places to See Before You Die  26.08   Five

Category: 
                            
          

In [206]:
# Part 3
#Finally, find and export into one list all the UPCs (under Product Information) 
# of books that have the word “Boys” in the title.
upcs = []

for cat in catagories:
    href = url + cat['href']
    
    while href:
        cat_resp = rq.get(href)
        cat_soup = bs(cat_resp.content, 'html.parser')

        cat_books = cat_soup.find_all('article', class_='product_pod')

        for book in cat_books:
            book_ref = book.h3.a['href'][9:]
            if 'boys' in book_ref.lower():
                book_url = url + 'catalogue/' + book.h3.a['href'][9:]
                book_rq = rq.get(book_url)
                book_soup = bs(book_rq.content, 'html.parser')
                
                # UPC is the first table element 
                product_info = book_soup.find('table', class_='table table-striped').td
                upcs.append(product_info.text)
        
        next_button = cat_soup.find('li', class_='next')
        if next_button:
            next_url = next_button.find('a')['href']
            href = urljoin(href, next_url)
        else:
            href = None


In [207]:
print(upcs)

['e10e1e165dc8be4a', 'd48c1369927ebef1', 'e0b3a679b83caccc', '61bc8167c9680283']
